# Neurite Targets Source Code

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt

In [1]:
import datajoint as dj
import numpy as np


print("neuron tab labeler started new")
#setting the address and the username
print("about to connect to database")
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'stelios'
dj.config['database.password'] = 'stelios#1'
dj.config['display.limit'] = 10
#will state whether words are shown or not
dj.config['safemode']=True
print(dj.conn(reset=True))


neuron tab labeler started new
about to connect to database
Connecting stelios@10.28.0.34:3306
DataJoint connection (connected) stelios@10.28.0.34:3306


In [2]:
schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
ndap100 = dj.create_virtual_module('ndap100', 'microns_ndap100')
fc = dj.create_virtual_module('fc', 'goldfunc_fc')

### Source presyns and postsyns

In [8]:
axon_segments = (ta3p100.Neurite() & 'neurite_type="axon"').proj(presyn='segment_id') * ta3p100.CurrentSegmentation()
post_trace_segments = (ta3p100.Segment() & ta3p100.CurrentSegmentation() & ndap100.Trace()).proj(postsyn='segment_id')

### Compartment Table

In [24]:
ta3p100.SynapseCompartmentLabel()

segmentation segmentation id,synapse_id synapse index within the segmentation,postsynaptic_label numeric label of the compartment
2,16650,1
2,69735,1
2,83755,1
2,89345,1
2,93929,1
2,144997,1
2,170880,1


In [12]:
final_synapses_with_comp_labels = ((ta3p100.Synapse() & post_trace_segments & axon_segments) - ta3p100.SynapseExclude().proj()) * ta3p100.SynapseCompartmentLabel
relevant_synapses_with_comp_labels = final_synapses_with_labels.proj("presyn","postsyn","postsynaptic_label")
# complete_synapse_comp_table = ((relevant_synapses_with_labels * ta3.LabelKey().proj('description', postsynaptic_label='numeric')).proj("presyn","postsynaptic_label","description"))

### Spine Table

In [15]:
ta3p100.SynapseSpineLabel()

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment
2,5,13
2,25,13
2,41,13
2,73,13
2,77,13
2,108,13
2,125,13


In [16]:
final_synapses_with_spine_labels = ((ta3p100.Synapse() & post_trace_segments & axon_segments) - ta3p100.SynapseExclude().proj()) * ta3p100.SynapseSpineLabel
relevant_synapses_with_spine_labels = final_synapses_with_spine_labels.proj("presyn","postsyn","spine_label")
complete_synapse_spine_table = ((relevant_synapses_with_spine_labels * ta3.LabelKey().proj('description', spine_label='numeric')).proj("presyn","spine_label","description"))

### Combine Label and Spine table

In [31]:
entries_to_add = dj.U('spine_label')*((dj.U('segmentation', 'synapse_id') & complete_synapse_comp_table) - (dj.U('segmentation', 'synapse_id') & complete_synapse_spine_table)).proj(spine_label='0')
updated_spine_table = complete_synapse_spine_table.proj() + entries_to_add.proj()
complete_spine_compartment_label_table = (relevant_synapses_with_comp_labels.proj('presyn', 'postsynaptic_label') * updated_spine_table).proj('presyn', compartment_label='postsynaptic_label')

In [47]:
complete_spine_compartment_label_table.heading

segmentation         : smallint                     # segmentation id
synapse_id           : bigint unsigned              # synapse index within the segmentation
spine_label          : tinyint unsigned             # numeric label of the compartment
---
presyn               : bigint unsigned              # segment id unique within each Segmentation
compartment_label    : tinyint unsigned             # numeric label of the compartment

In [76]:
@schema
class SynapseCompartmentSpineLabel(dj.Manual):
    definition = """
    -> ta3p100.Synapse
    segmentation         : smallint                     # segmentation id
    synapse_id           : bigint unsigned              # synapse index within the segmentation
    ---
    spine_label          : tinyint unsigned             # numeric label of the compartment
    segment_id               : bigint unsigned              # segment id unique within each Segmentation
    compartment_label    : tinyint unsigned             # numeric label of the compartment
    """

In [94]:
SynapseCompartmentSpineLabel().heading

# 
segmentation         : smallint                     # segmentation id
synapse_id           : bigint unsigned              # synapse index within the segmentation
---
spine_label          : tinyint unsigned             # numeric label of the compartment
segment_id           : bigint unsigned              # segment id unique within each Segmentation
compartment_label    : tinyint unsigned             # numeric label of the compartment

In [56]:
ta3p100.LabelKey() #& "description='Spine Neck'"

numeric numeric label of the compartment,description descriptive name of the label,color representative color of the compartment
0,not_labeled,no_color
1,label_removed,no_color
2,Apical,blue
3,Basal,yellow
4,Oblique,green
5,Soma,red
6,Axon-Soma,aqua


In [77]:
SynapseCompartmentSpineLabel.insert(complete_spine_compartment_label_table.proj("compartment_label",segment_id="presyn"))




In [84]:
SynapseCompartmentSpineLabel().heading

# 
segmentation         : smallint                     # segmentation id
synapse_id           : bigint unsigned              # synapse index within the segmentation
---
spine_label          : tinyint unsigned             # numeric label of the compartment
segment_id           : bigint unsigned              # segment id unique within each Segmentation
compartment_label    : tinyint unsigned             # numeric label of the compartment

In [ ]:
#spine = 14
#spine head = 13
#spine neck = 15
#spine shat

In [59]:
print(ta3p100.LabelKey().fetch())

[( 0, 'not_labeled', 'no_color') ( 1, 'label_removed', 'no_color')
 ( 2, 'Apical', 'blue') ( 3, 'Basal', 'yellow') ( 4, 'Oblique', 'green')
 ( 5, 'Soma', 'red') ( 6, 'Axon-Soma', 'aqua')
 ( 7, 'Axon-Dendr', 'off blue') ( 8, 'Dendrite', 'purple')
 ( 9, 'Distal', 'pink') (10, 'Error', 'brown') (11, 'Unlabelable', 'tan')
 (12, 'Cilia', 'light purple') (13, 'Spine Head', 'rose')
 (14, 'Spine', 'light pink') (15, 'Spine Neck', 'light green')
 (16, 'Bouton', 'aqua')]


In [86]:
fingerprints = dj.U('segmentation','segment_id').aggr((SynapseCompartmentSpineLabel), 
                    apical_num='sum(compartment_label = 2)',
                    apical_prop='sum(compartment_label = 2)/count(*)',
                    apical_spine_prop='sum(spine_label = 14 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_spine_head_prop='sum(spine_label = 13 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_shaft_prop='sum(spine_label = 0 AND compartment_label = 2)/count(compartment_label = 2)',  
                                   
                    basal_num='sum(compartment_label = 3)',
                    basal_prop='sum(compartment_label = 3)/count(*)',
                    basal_spine_prop='sum(spine_label = 14 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_spine_head_prop='sum(spine_label = 13 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_shaft_prop='sum(spine_label = 0 AND compartment_label = 3)/count(compartment_label = 3)', 
                    
                    oblique_num='sum(compartment_label = 4)',
                    oblique_prop='sum(compartment_label = 4)/count(*)',
                    oblique_spine_prop='sum(spine_label = 14 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_spine_head_prop='sum(spine_label = 13 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_shaft_prop='sum(spine_label = 0 AND compartment_label = 4)/count(compartment_label = 4)', 
                                   
                    soma_num='sum(compartment_label = 5)',
                    soma_prop='sum(compartment_label = 5)/count(*)',
                    soma_spine_prop='sum(spine_label = 14 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_spine_head_prop='sum(spine_label = 13 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_shaft_prop='sum(spine_label = 0 AND compartment_label = 5)/count(compartment_label = 5)', 
                                   
                    axon_num='sum(compartment_label = 6 OR spine_label = 7)',
                    axon_prop='sum(compartment_label = 6 OR spine_label = 7)/count(*)',
                    
                    spine_num='sum(spine_label = 14)',
                    spine_prop='sum(spine_label = 14)/count(*)',
                    spine_head_num='sum(spine_label = 13)',
                    spine_head_prop='sum(spine_label = 13)/count(*)',
                    spine_neck_num='sum(spine_label = 15)',
                    spine_neck_prop='sum(spine_label = 15)/count(*)',
                    shaft_num='sum(spine_label = 0)',
                    shaft_prop='sum(spine_label = 0)/count(*)',
                    total='count(*)')
fingerprints

segment_id segment id unique within each Segmentation,oblique_prop calculated attribute,axon_prop calculated attribute,total calculated attribute,soma_spine_neck_prop calculated attribute,apical_spine_prop calculated attribute,soma_num calculated attribute,shaft_prop calculated attribute,oblique_spine_neck_prop calculated attribute,basal_num calculated attribute,oblique_spine_prop calculated attribute,spine_neck_prop calculated attribute,basal_prop calculated attribute,shaft_num calculated attribute,apical_spine_neck_prop calculated attribute,soma_spine_prop calculated attribute,soma_prop calculated attribute,apical_shaft_prop calculated attribute,basal_spine_prop calculated attribute,oblique_spine_head_prop calculated attribute,spine_num calculated attribute,axon_num calculated attribute,oblique_num calculated attribute,basal_spine_head_prop calculated attribute,spine_prop calculated attribute,oblique_shaft_prop calculated attribute,basal_spine_neck_prop calculated attribute,soma_spine_head_prop calculated attribute,apical_prop calculated attribute,apical_spine_head_prop calculated attribute,apical_num calculated attribute,soma_shaft_prop calculated attribute,spine_neck_num calculated attribute,spine_head_prop calculated attribute,spine_head_num calculated attribute,basal_shaft_prop calculated attribute
648518346341351467,0.0000,0.0000,3,0.0000,0.0000,0,0.0000,0.0000,3,0.0000,0.3333,1.0000,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0.6667,0.0000,0.0000,0.3333,0.0000,0.0000,0.0000,0,0.0000,1,0.6667,2,0.0000
648518346341351503,0.0000,0.0000,3,0.0000,0.0000,0,0.0000,0.0000,3,0.0000,0.0000,1.0000,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0,1.0000,3,0.0000
648518346341351508,0.0000,0.0000,4,0.0000,0.0000,2,0.7500,0.0000,2,0.0000,0.0000,0.5000,3,0.0000,0.0000,0.5000,0.0000,0.0000,0.0000,0,0,0,0.2500,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.5000,0,0.2500,1,0.2500
648518346341351509,0.3333,0.0000,6,0.0000,0.0000,0,0.1667,0.0000,3,0.0000,0.0000,0.5000,1,0.0000,0.0000,0.0000,0.0000,0.1667,0.3333,1,0,2,0.3333,0.1667,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0,0.6667,4,0.0000
648518346341351512,0.0000,0.0000,6,0.0000,0.0000,0,0.3333,0.0000,6,0.0000,0.0000,1.0000,2,0.0000,0.0000,0.0000,0.0000,0.1667,0.0000,1,0,0,0.5000,0.1667,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0,0.5000,3,0.3333
648518346341351514,0.0000,0.0000,6,0.0000,0.0000,0,0.3333,0.0000,6,0.0000,0.0000,1.0000,2,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0.6667,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0,0.6667,4,0.3333
648518346341351518,0.0000,0.0000,4,0.0000,0.0000,0,0.0000,0.0000,4,0.0000,0.0000,1.0000,0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0.0000,0,1.0000,4,0.0000
648518346341351523,0.0000,0.0000,3,0.0000,0.0000,0,0.3333,0.0000,2,0.0000,0.0000,0.6667,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0.3333,0.0000,0.0000,0.0000,0.0000,0.3333,0.3333,1,0.0000,0,0.6667,2,0.3333
648518346341351525,0.0000,0.0000,9,0.0000,0.0000,0,0.0000,0.0000,9,0.0000,0.1111,1.0000,0,0.0000,0.0000,0.0000,0.0000,0.2222,0.0000,2,0,0,0.6667,0.2222,0.0000,0.1111,0.0000,0.0000,0.0000,0,0.0000,1,0.6667,6,0.0000
648518346341351531,0.0000,0.0000,4,0.0000,0.0000,0,0.2500,0.0000,4,0.0000,0.2500,1.0000,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0,0,0,0.5000,0.0000,0.0000,0.2500,0.0000,0.0000,0.0000,0,0.0000,1,0.5000,2,0.2500


In [92]:
fingerprints.heading

segment_id           : bigint unsigned              # segment id unique within each Segmentation
---
oblique_prop         : expression                   # calculated attribute
axon_prop            : expression                   # calculated attribute
total                : expression                   # calculated attribute
soma_spine_neck_prop : expression                   # calculated attribute
apical_spine_prop    : expression                   # calculated attribute
soma_num             : expression                   # calculated attribute
shaft_prop           : expression                   # calculated attribute
oblique_spine_neck_prop : expression                   # calculated attribute
basal_num            : expression                   # calculated attribute
oblique_spine_prop   : expression                   # calculated attribute
spine_neck_prop      : expression                   # calculated attribute
basal_prop           : expression                   # calculated attrib

In [ ]:
 oblique_spine_prop    : float           #% of oblique synapses that are oblique spine synapses 
    oblique_spine_head_prop    : float           #% of oblique synapses that are oblique spine head synapses 
    oblique_spine_neck_prop    : float           #% of oblique synapses that are oblique spine neck synapses 
    oblique_shaft_prop    : float           #% of oblique synapses that are oblique shaft synapses 
    soma_num    : float           #total number of soma synapsis for neurite
    soma_prop    : float           #% of total number of soma synapsis for neurite
    soma_spine_prop    : float           #% of soma synapses that are soma spine synapses 
    soma_spine_head_prop    : float           #% of soma synapses that are soma spine head synapses 
    soma_spine_neck_prop    : float           #% of soma synapses that are soma spine neck synapses 
    soma_shaft_prop    : float           #% of soma synapses that are soma shaft synapses 
    axon_num              : float           #total number of axon synapsis for neurite
    axon_prop             : float           #% of total number of axon synapsis for neurite
    spine_num             : float           #total number of spine synapsis for neurite
    spine_prop            : float           #% of total number of spine synapsis for neurite
    spine_head_num        : float           #total number of spine head synapsis for neurite
    spine_head_prop       : float           #% of total number of spine head synapsis for neurite
    spine_neck_num        : float           #total number of spine neck synapsis for neurite
    spine_neck_prop       : float           #% of total number of spine neck synapsis for neurite
    shaft_num             : float           #total number of shaft synapsis for neurite
    shaft_prop            : float           #% of total number of shaft synapsis for neurite
    total                 : float           #total number of synapsis for neurite
   

In [ ]:
    apical_num    : float           # total number of apicals synapsis for neurite
    apical_prop    : float           # perc of total number of apicals synapsis for neurite
    apical_spine_prop    : float           # perc of apical synapses that are apical spine synapses 
    apical_spine_head_prop    : float           #perc of apical synapses that are apical spine head synapses 
    apical_spine_neck_prop    : float           #perc of apical synapses that are apical spine neck synapses 
    apical_shaft_prop    : float           #perc of apical synapses that are apical shaft synapses 
    basal_num    : float           #total number of basal synapsis for neurite
    basal_prop    : float           #perc of total number of basal synapsis for neurite
    basal_spine_prop    : float           #perc of basal synapses that are basal spine synapses 
    basal_spine_head_prop    : float           #perc of basal synapses that are basal spine head synapses 
    basal_spine_neck_prop    : float           #perc of basal synapses that are basal spine neck synapses 
    basal_shaft_prop    : float           #perc of basal synapses that are basal shaft synapses 
    oblique_num    : float           #total number of oblique synapsis for neurite
    oblique_prop    : float           #perc of total number of oblique synapsis for neurite
    oblique_spine_prop    : float           #perc of oblique synapses that are oblique spine synapses 
    oblique_spine_head_prop    : float           #perc of oblique synapses that are oblique spine head synapses 
    oblique_spine_neck_prop    : float           #perc of oblique synapses that are oblique spine neck synapses 
    oblique_shaft_prop    : float           #perc of oblique synapses that are oblique shaft synapses 
    soma_num    : float           #total number of soma synapsis for neurite
    soma_prop    : float           #perc of total number of soma synapsis for neurite
    soma_spine_prop    : float           #perc of soma synapses that are soma spine synapses 
    soma_spine_head_prop    : float           #perc of soma synapses that are soma spine head synapses 
    soma_spine_neck_prop    : float           #perc of soma synapses that are soma spine neck synapses 
    soma_shaft_prop       : float           #perc of soma synapses that are soma shaft synapses 
    axon_num              : float           #total number of axon synapsis for neurite
    axon_prop             : float           #perc of total number of axon synapsis for neurite
    spine_num             : float           #total number of spine synapsis for neurite
    spine_prop            : float           #perc of total number of spine synapsis for neurite
    spine_head_num        : float           #total number of spine head synapsis for neurite
    spine_head_prop       : float           #perc of total number of spine head synapsis for neurite
    spine_neck_num        : float           #total number of spine neck synapsis for neurite
    spine_neck_prop       : float           #perc of total number of spine neck synapsis for neurite
    shaft_num             : float           #total number of shaft synapsis for neurite
    shaft_prop            : float           #perc of total number of shaft synapsis for neurite
    total                 : float           #total number of synapsis for neurite

In [115]:
schema

Schema `microns_ta3p100`

In [111]:
@schema
class NeuriteTargets(dj.Computed):
    definition="""
    ->ta3p100.Neurite
    ---
    apical_num    : float           # total number of apicals synapsis for neurite
    apical_prop    : float           # perc of total number of apicals synapsis for neurite
    apical_spine_prop    : float           # perc of apical synapses that are apical spine synapses 
    apical_spine_head_prop    : float           #perc of apical synapses that are apical spine head synapses 
    apical_spine_neck_prop    : float           #perc of apical synapses that are apical spine neck synapses 
    apical_shaft_prop    : float           #perc of apical synapses that are apical shaft synapses 
    basal_num    : float           #total number of basal synapsis for neurite
    basal_prop    : float           #perc of total number of basal synapsis for neurite
    basal_spine_prop    : float           #perc of basal synapses that are basal spine synapses 
    basal_spine_head_prop    : float           #perc of basal synapses that are basal spine head synapses 
    basal_spine_neck_prop    : float           #perc of basal synapses that are basal spine neck synapses 
    basal_shaft_prop    : float           #perc of basal synapses that are basal shaft synapses 
    oblique_num    : float           #total number of oblique synapsis for neurite
    oblique_prop    : float           #perc of total number of oblique synapsis for neurite
    oblique_spine_prop    : float           #perc of oblique synapses that are oblique spine synapses 
    oblique_spine_head_prop    : float           #perc of oblique synapses that are oblique spine head synapses 
    oblique_spine_neck_prop    : float           #perc of oblique synapses that are oblique spine neck synapses 
    oblique_shaft_prop    : float           #perc of oblique synapses that are oblique shaft synapses 
    soma_num    : float           #total number of soma synapsis for neurite
    soma_prop    : float           #perc of total number of soma synapsis for neurite
    soma_spine_prop    : float           #perc of soma synapses that are soma spine synapses 
    soma_spine_head_prop    : float           #perc of soma synapses that are soma spine head synapses 
    soma_spine_neck_prop    : float           #perc of soma synapses that are soma spine neck synapses 
    soma_shaft_prop       : float           #perc of soma synapses that are soma shaft synapses 
    axon_num              : float           #total number of axon synapsis for neurite
    axon_prop             : float           #perc of total number of axon synapsis for neurite
    spine_num             : float           #total number of spine synapsis for neurite
    spine_prop            : float           #perc of total number of spine synapsis for neurite
    spine_head_num        : float           #total number of spine head synapsis for neurite
    spine_head_prop       : float           #perc of total number of spine head synapsis for neurite
    spine_neck_num        : float           #total number of spine neck synapsis for neurite
    spine_neck_prop       : float           #perc of total number of spine neck synapsis for neurite
    shaft_num             : float           #total number of shaft synapsis for neurite
    shaft_prop            : float           #perc of total number of shaft synapsis for neurite
    total                 : float           #total number of synapsis for neurite
    """
    
    key_source = ta3p100.CurrentSegmentation()
    
    def make(self, key):
        fingerprints = dj.U('segmentation','segment_id').aggr((SynapseCompartmentSpineLabel & key), 
                    apical_num='sum(compartment_label = 2)',
                    apical_prop='sum(compartment_label = 2)/count(*)',
                    apical_spine_prop='sum(spine_label = 14 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_spine_head_prop='sum(spine_label = 13 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_shaft_prop='sum(spine_label = 0 AND compartment_label = 2)/count(compartment_label = 2)',  
                                   
                    basal_num='sum(compartment_label = 3)',
                    basal_prop='sum(compartment_label = 3)/count(*)',
                    basal_spine_prop='sum(spine_label = 14 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_spine_head_prop='sum(spine_label = 13 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_shaft_prop='sum(spine_label = 0 AND compartment_label = 3)/count(compartment_label = 3)', 
                    
                    oblique_num='sum(compartment_label = 4)',
                    oblique_prop='sum(compartment_label = 4)/count(*)',
                    oblique_spine_prop='sum(spine_label = 14 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_spine_head_prop='sum(spine_label = 13 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_shaft_prop='sum(spine_label = 0 AND compartment_label = 4)/count(compartment_label = 4)', 
                                   
                    soma_num='sum(compartment_label = 5)',
                    soma_prop='sum(compartment_label = 5)/count(*)',
                    soma_spine_prop='sum(spine_label = 14 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_spine_head_prop='sum(spine_label = 13 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_shaft_prop='sum(spine_label = 0 AND compartment_label = 5)/count(compartment_label = 5)', 
                                   
                    axon_num='sum(compartment_label = 6 OR spine_label = 7)',
                    axon_prop='sum(compartment_label = 6 OR spine_label = 7)/count(*)',
                    
                    spine_num='sum(spine_label = 14)',
                    spine_prop='sum(spine_label = 14)/count(*)',
                    spine_head_num='sum(spine_label = 13)',
                    spine_head_prop='sum(spine_label = 13)/count(*)',
                    spine_neck_num='sum(spine_label = 15)',
                    spine_neck_prop='sum(spine_label = 15)/count(*)',
                    shaft_num='sum(spine_label = 0)',
                    shaft_prop='sum(spine_label = 0)/count(*)',
                    total='count(*)')
        
        self.insert(fingerprints)

In [7]:
plt.hist(spine_labels)

NameError: name 'plt' is not defined

In [116]:
NeuriteTargets()

segmentation segmentation id,segment_id segment id unique within each Segmentation,apical_num total number of apicals synapsis for neurite,apical_prop perc of total number of apicals synapsis for neurite,apical_spine_prop perc of apical synapses that are apical spine synapses,apical_spine_head_prop perc of apical synapses that are apical spine head synapses,apical_spine_neck_prop perc of apical synapses that are apical spine neck synapses,apical_shaft_prop perc of apical synapses that are apical shaft synapses,basal_num total number of basal synapsis for neurite,basal_prop perc of total number of basal synapsis for neurite,basal_spine_prop perc of basal synapses that are basal spine synapses,basal_spine_head_prop perc of basal synapses that are basal spine head synapses,basal_spine_neck_prop perc of basal synapses that are basal spine neck synapses,basal_shaft_prop perc of basal synapses that are basal shaft synapses,oblique_num total number of oblique synapsis for neurite,oblique_prop perc of total number of oblique synapsis for neurite,oblique_spine_prop perc of oblique synapses that are oblique spine synapses,oblique_spine_head_prop perc of oblique synapses that are oblique spine head synapses,oblique_spine_neck_prop perc of oblique synapses that are oblique spine neck synapses,oblique_shaft_prop perc of oblique synapses that are oblique shaft synapses,soma_num total number of soma synapsis for neurite,soma_prop perc of total number of soma synapsis for neurite,soma_spine_prop perc of soma synapses that are soma spine synapses,soma_spine_head_prop perc of soma synapses that are soma spine head synapses,soma_spine_neck_prop perc of soma synapses that are soma spine neck synapses,soma_shaft_prop perc of soma synapses that are soma shaft synapses,axon_num total number of axon synapsis for neurite,axon_prop perc of total number of axon synapsis for neurite,spine_num total number of spine synapsis for neurite,spine_prop perc of total number of spine synapsis for neurite,spine_head_num total number of spine head synapsis for neurite,spine_head_prop perc of total number of spine head synapsis for neurite,spine_neck_num total number of spine neck synapsis for neurite,spine_neck_prop perc of total number of spine neck synapsis for neurite,shaft_num total number of shaft synapsis for neurite,shaft_prop perc of total number of shaft synapsis for neurite,total total number of synapsis for neurite
2,648518346341351467,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.666667,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.666667,1.0,0.333333,0.0,0.0,3.0
2,648518346341351503,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,3.0
2,648518346341351508,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.5,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,1.0,0.25,0.0,0.0,3.0,0.75,4.0
2,648518346341351509,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.5,0.166667,0.333333,0.0,0.0,2.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667,4.0,0.666667,0.0,0.0,1.0,0.166667,6.0
2,648518346341351512,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,0.166667,0.5,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667,3.0,0.5,0.0,0.0,2.0,0.333333,6.0
2,648518346341351514,0.0,0.0,0.0,0.0,0.0,0.0,6.0,1.0,0.0,0.666667,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.666667,0.0,0.0,2.0,0.333333,6.0
2,648518346341351518,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,4.0
2,648518346341351523,1.0,0.333333,0.0,0.333333,0.0,0.0,2.0,0.666667,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.666667,0.0,0.0,1.0,0.333333,3.0
2,648518346341351525,0.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,0.222222,0.666667,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.2

In [98]:
fingerprints = dj.U('segmentation','segment_id').aggr((SynapseCompartmentSpineLabel), 
                    apical_num='sum(compartment_label = 2)',
                    apical_prop='sum(compartment_label = 2)/count(*)',
                    apical_spine_prop='sum(spine_label = 14 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_spine_head_prop='sum(spine_label = 13 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 2)/count(compartment_label = 2)',
                    apical_shaft_prop='sum(spine_label = 0 AND compartment_label = 2)/count(compartment_label = 2)',  
                                   
                    basal_num='sum(compartment_label = 3)',
                    basal_prop='sum(compartment_label = 3)/count(*)',
                    basal_spine_prop='sum(spine_label = 14 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_spine_head_prop='sum(spine_label = 13 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 3)/count(compartment_label = 3)',
                    basal_shaft_prop='sum(spine_label = 0 AND compartment_label = 3)/count(compartment_label = 3)', 
                    
                    oblique_num='sum(compartment_label = 4)',
                    oblique_prop='sum(compartment_label = 4)/count(*)',
                    oblique_spine_prop='sum(spine_label = 14 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_spine_head_prop='sum(spine_label = 13 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 4)/count(compartment_label = 4)',
                    oblique_shaft_prop='sum(spine_label = 0 AND compartment_label = 4)/count(compartment_label = 4)', 
                                   
                    soma_num='sum(compartment_label = 5)',
                    soma_prop='sum(compartment_label = 5)/count(*)',
                    soma_spine_prop='sum(spine_label = 14 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_spine_head_prop='sum(spine_label = 13 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_spine_neck_prop='sum(spine_label = 15 AND compartment_label = 5)/count(compartment_label = 5)',
                    soma_shaft_prop='sum(spine_label = 0 AND compartment_label = 5)/count(compartment_label = 5)', 
                                   
                    axon_num='sum(compartment_label = 6 OR spine_label = 7)',
                    axon_prop='sum(compartment_label = 6 OR spine_label = 7)/count(*)',
                    
                    spine_num='sum(spine_label = 14)',
                    spine_prop='sum(spine_label = 14)/count(*)',
                    spine_head_num='sum(spine_label = 13)',
                    spine_head_prop='sum(spine_label = 13)/count(*)',
                    spine_neck_num='sum(spine_label = 15)',
                    spine_neck_prop='sum(spine_label = 15)/count(*)',
                    shaft_num='sum(spine_label = 0)',
                    shaft_prop='sum(spine_label = 0)/count(*)',
                    total='count(*)')

In [99]:
fingerprints.heading

segmentation         : smallint                     # segmentation id
segment_id           : bigint unsigned              # segment id unique within each Segmentation
---
oblique_prop         : expression                   # calculated attribute
axon_prop            : expression                   # calculated attribute
total                : expression                   # calculated attribute
soma_spine_neck_prop : expression                   # calculated attribute
apical_spine_prop    : expression                   # calculated attribute
soma_num             : expression                   # calculated attribute
shaft_prop           : expression                   # calculated attribute
oblique_spine_neck_prop : expression                   # calculated attribute
basal_num            : expression                   # calculated attribute
oblique_spine_prop   : expression                   # calculated attribute
spine_neck_prop      : expression                   # calculated attribute
b

In [82]:
SynapseCompartmentSpineLabel() & ta3p100.CurrentSegmentation()

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment,segment_id segment id unique within each Segmentation,compartment_label numeric label of the compartment
2,40,0,648518346342791483,2
2,73,13,648518346341407883,3
2,137,13,648518346346598367,3
2,262,0,648518346341369116,2
2,295,15,648518346342930039,3
2,342,13,648518346342921993,2
2,352,13,648518346341393750,3
2,360,15,648518346341375326,3
2,362,0,648518346349386578,5
2,412,13,648518346341419530,2


In [64]:
dj.U("presyn") & (((ta3p100.Synapse() & post_trace_segments & axon_segments) - ta3p100.SynapseExclude().proj()) * ta3p100.SynapseCompartmentLabel)

presyn segment id unique within each Segmentation
648518346341351467
648518346341351503
648518346341351508
648518346341351509
648518346341351512
648518346341351514
648518346341351518


In [61]:
ta3p100.Neurite()

segmentation segmentation id,segment_id segment id unique within each Segmentation,neurite_type neurite type
2,648518346341351467,axon
2,648518346341351503,axon
2,648518346341351508,axon
2,648518346341351509,axon
2,648518346341351512,axon
2,648518346341351514,axon
2,648518346341351518,axon


In [ ]:
fingerprints = dj.U('presyn').aggr((relevant_axon_trace_synapses_spine), 
                    spine_num='sum(description = "Spine")',
                    spine_prop='sum(description = "Spine")/count(*)',
                    spine_head='sum(description = "Spine Head")',
                    spine_head_prop='sum(description = "Spine Head")/count(*)',
                    spine_neck='sum(description = "Spine Neck")',
                    spine_neck_prop='sum(description = "Spine Neck")/count(*)',
                    total='count(*)') & 'total>0'

In [17]:
complete_synapse_spine_table

segmentation segmentation id,synapse_id synapse index within the segmentation,spine_label numeric label of the compartment,presyn segment id unique within each Segmentation,description descriptive name of the label
2,1129926,13,648518346342466506,Spine Head
2,3248142,13,648518346342790055,Spine Head
2,2744287,13,648518346342792404,Spine Head
2,1782945,13,648518346342794433,Spine Head
2,3445483,13,648518346341402579,Spine Head
2,1139077,13,648518346341404783,Spine Head
2,151516,13,648518346341407137,Spine Head


In [11]:
"""   HOW THE SharedInput Table is generated
#how they generate the SharedInput
soma = ta3.Segment & nda.Trace
axons = ta3.Neurite & 'neurite_type="axon"'
synapse = ta3.Synapse & axons.proj(presyn='segment_id') & complete_synapse_table_fixed
info = (soma * soma.proj(segment_b='segment_id') & 'segment_id > segment_b' 
                & complete_synapse_table_fixed.proj(segment_b="postsyn") 
                & complete_synapse_table_fixed.proj(segment_id="postsyn"))
A = (synapse & soma.proj(postsyn='segment_id')).proj(
    'presyn', syn1='synapse_id', segment_id='postsyn')
B = (synapse * dj.U('presyn') & soma.proj(postsyn='segment_id')).proj(
    'presyn', syn2='synapse_id', segment_b='postsyn')
shared = dj.U('segment_id', 'segment_b').aggr(A * B & 'segment_id > segment_b',
                                              n_seg_shared='count(DISTINCT presyn)')



shared
other_half_table = #info - shared """

'   HOW THE SharedInput Table is generated\n#how they generate the SharedInput\nsoma = ta3.Segment & nda.Trace\naxons = ta3.Neurite & \'neurite_type="axon"\'\nsynapse = ta3.Synapse & axons.proj(presyn=\'segment_id\') & complete_synapse_table_fixed\ninfo = (soma * soma.proj(segment_b=\'segment_id\') & \'segment_id > segment_b\' \n                & complete_synapse_table_fixed.proj(segment_b="postsyn") \n                & complete_synapse_table_fixed.proj(segment_id="postsyn"))\nA = (synapse & soma.proj(postsyn=\'segment_id\')).proj(\n    \'presyn\', syn1=\'synapse_id\', segment_id=\'postsyn\')\nB = (synapse * dj.U(\'presyn\') & soma.proj(postsyn=\'segment_id\')).proj(\n    \'presyn\', syn2=\'synapse_id\', segment_b=\'postsyn\')\nshared = dj.U(\'segment_id\', \'segment_b\').aggr(A * B & \'segment_id > segment_b\',\n                                              n_seg_shared=\'count(DISTINCT presyn)\')\n\n\n\nshared\nother_half_table = #info - shared '

In [12]:
#pull down the data
synapse_id,postsyn,postsynaptic_label,description,presyn = complete_synapse_table.fetch("synapse_id",
                                                        "postsyn","postsynaptic_label","description","presyn")



In [13]:
#add on the spine labels
synapse_id_spine, spine_label = ta3.SynapseSpineLabel().fetch("synapse_id","spine_label")



In [14]:
label_key = ta3.LabelKey().fetch("description")
label_key[0]

'not_labeled'

In [15]:
from tqdm import tqdm
#create lookup index
spine_lookup=dict()
for syn_id, syn_label in tqdm(zip(synapse_id_spine,spine_label)):
    spine_lookup[syn_id] = syn_label
    
#find out who is not in the list
#print(spine_lookup)

134077it [00:00, 185721.14it/s]


In [16]:

#create the extra spine list
postsynaptic_label_spine = list()
spine_description = list()
spine_label_lookup = dict()
synapse_id_spine = set(synapse_id_spine)
for i,syn in tqdm(enumerate(synapse_id)):
    if syn in synapse_id_spine:
        #print("inside labeler")
        label = spine_lookup[syn]
    else:
        label = 0
        
    spine_label_lookup[syn] = label
    postsynaptic_label_spine.append(label)
    spine_description.append(label_key[label]) 

        
        

173555it [00:01, 151510.26it/s]


In [17]:
#create pandas table from all of the data
import pandas as pd
postsynaptic_label_spine

synapse_id,postsyn,postsynaptic_label,description,presyn
d = dict(synapse_id=synapse_id,presyn=presyn,postsyn=postsyn,postsynaptic_label=postsynaptic_label,
         spine_numeric=postsynaptic_label_spine,description=description,spine_description=spine_description
          )

Label_Table = pd.DataFrame(d)
Label_Table



,synapse_id,presyn,postsyn,postsynaptic_label,spine_numeric,description,spine_description
0,21460,54101819,36456160,3,0,Basal,not_labeled
1,21485,50165577,36456160,3,14,Basal,Spine
2,23369,58116980,36456160,3,15,Basal,Spine Neck
3,23579,54037914,36456160,3,15,Basal,Spine Neck
4,23689,54101819,36456160,3,0,Basal,not_labeled
5,23728,54034799,36456160,3,13,Basal,Spine Head
6,23805,58036847,36456160,3,13,Basal,Spine Head
7,23870,50165937,36456160,3,0,Basal,not_labeled
8,25181,58037816,36456160,3,14,Basal,Spine
9,25797,31523985,36456160,3,14,Basal,Spine


In [18]:
#query the datatable for list
possible_spines = ["Spine","Spine Neck","Spine Head"]
#possible_spines = ["not_labeled"]
Label_Table.loc[Label_Table["spine_description"].isin(possible_spines)]




,synapse_id,presyn,postsyn,postsynaptic_label,spine_numeric,description,spine_description
1,21485,50165577,36456160,3,14,Basal,Spine
2,23369,58116980,36456160,3,15,Basal,Spine Neck
3,23579,54037914,36456160,3,15,Basal,Spine Neck
5,23728,54034799,36456160,3,13,Basal,Spine Head
6,23805,58036847,36456160,3,13,Basal,Spine Head
8,25181,58037816,36456160,3,14,Basal,Spine
9,25797,31523985,36456160,3,14,Basal,Spine
10,26354,58045292,36456160,3,13,Basal,Spine Head
11,26431,62023047,36456160,3,13,Basal,Spine Head
12,26702,58053252,36456160,3,13,Basal,Spine Head


In [19]:
#find the synapses without the Spines
no_spine_table = Label_Table.loc[Label_Table["spine_description"].isin(["not_labeled","Spine Neck","Spine Head"])]
no_spine_table

,synapse_id,presyn,postsyn,postsynaptic_label,spine_numeric,description,spine_description
0,21460,54101819,36456160,3,0,Basal,not_labeled
2,23369,58116980,36456160,3,15,Basal,Spine Neck
3,23579,54037914,36456160,3,15,Basal,Spine Neck
4,23689,54101819,36456160,3,0,Basal,not_labeled
5,23728,54034799,36456160,3,13,Basal,Spine Head
6,23805,58036847,36456160,3,13,Basal,Spine Head
7,23870,50165937,36456160,3,0,Basal,not_labeled
10,26354,58045292,36456160,3,13,Basal,Spine Head
11,26431,62023047,36456160,3,13,Basal,Spine Head
12,26702,58053252,36456160,3,13,Basal,Spine Head


In [20]:
from collections import Counter
#pull out of the dataframe, compute the statistics and then put back in the table
#make sure these are numpy arrays or else the where function won't work
synapse_ids = no_spine_table['synapse_id']
presyns = Label_Table['presyn']
postsyns = Label_Table['postsyn']
synapse_labels = Label_Table['postsynaptic_label']
synapse_labels_spine = no_spine_table['spine_numeric']
description = Label_Table['description']
description_spine = Label_Table['spine_description']



''' Way Christos aggregates:
1) Creates counter for pre and post
2) creates dictionary that maps the postsynaptic target to the synapse
3) creates dictionary for synap --> label
    create reg label, joint label, and spine label
4) maps axons --> synapse
'''


"""
#aggregate the data
postsyns_to_synapses = dict()
for postsyn in tqdm(postsyns_counter):
    postsyns_to_synapses[postsyn] = synapse_ids[np.where(postsyns==postsyn)[0]]
    
#creates dictionary for synap --> label
synapse_label_dict = dict()
for synapse_id, label in tqdm(zip(*synapse_labels)):
    synapse_label_dict[synapse_id] = label

#maps axons --> synapse
axon_to_synapse = dict()
for synapse, presyn in tqdm(zip(synapse_ids, presyns)):
    if presyn in axon_to_synapse:
        axon_to_synapse[presyn].append(synapse)
    else:
        axon_to_synapse[presyn] = list()
        axon_to_synapse[presyn].append(synapse)
        
"""

'\n#aggregate the data\npostsyns_to_synapses = dict()\nfor postsyn in tqdm(postsyns_counter):\n    postsyns_to_synapses[postsyn] = synapse_ids[np.where(postsyns==postsyn)[0]]\n    \n#creates dictionary for synap --> label\nsynapse_label_dict = dict()\nfor synapse_id, label in tqdm(zip(*synapse_labels)):\n    synapse_label_dict[synapse_id] = label\n\n#maps axons --> synapse\naxon_to_synapse = dict()\nfor synapse, presyn in tqdm(zip(synapse_ids, presyns)):\n    if presyn in axon_to_synapse:\n        axon_to_synapse[presyn].append(synapse)\n    else:\n        axon_to_synapse[presyn] = list()\n        axon_to_synapse[presyn].append(synapse)\n        \n'

In [21]:
compartment_labels = Counter(description).keys()
compartment_labels

dict_keys(['Basal', 'Soma', 'Axon-Soma', 'Oblique', 'Apical', 'Error', 'Axon-Dendr', 'Unlabelable', 'Dendrite'])

In [22]:
#gets the counter for presynapses and postysnaptic
presyns_counter, postsyns_counter = Counter(presyns), Counter(postsyns)
#print(presyns_counter)

#creates dictionary that maps the postsynaptic target to the synapse

postsyns_to_synapses = dict()
for postsyn in tqdm(postsyns_counter.keys()):
    postsyns_to_synapses[postsyn] = synapse_ids[np.where(postsyns==postsyn)[0]]
    
#creates dictionary for synap --> label
synapse_label_dict = dict()
for synapse_id, label in tqdm(zip(synapse_ids,synapse_labels)):
    synapse_label_dict[synapse_id] = label

#creates dictionary for synap --> spine label
synapse_label_dict_spine = dict()
for synapse_id, label in tqdm(zip(synapse_ids,synapse_labels_spine)):
    synapse_label_dict_spine[synapse_id] = label
    
#DON'T THINK NEED JOINT LABELS!!
#maps axons --> synapse
axon_to_synapse = dict()
for synapse, presyn in tqdm(zip(synapse_ids, presyns)):
    if presyn in axon_to_synapse:
        axon_to_synapse[presyn].append(synapse)
    else:
        axon_to_synapse[presyn] = list()
        axon_to_synapse[presyn].append(synapse)



  0%|          | 0/167 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:842: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
100%|██████████| 167/167 [00:00<00:00, 214.12it/s]
159960it [00:00, 190516.30it/s]
159960it [00:00, 193899.19it/s]
159960it [00:01, 136673.66it/s]


In [23]:
#The destination vectors that will be fed into the clustering machine

#WHERE COMPARTMENT LABELS AND SPINES NOT SEPERATE

axon_to_vector = dict()
threshold_axon_to_vector = dict()
axon_to_vector_spine = dict()
threshold_axon_to_vector_spine = dict()

axon_to_vector_spine_joint = dict()
threshold_axon_to_vector_spine_joint = dict()



#aggregating the data
for presyn in tqdm(axon_to_synapse):
    label_list = list()
    label_list_spine = list()
    #list to hold both compartments
    joint_list = list()
    for synapse in axon_to_synapse[presyn]:
        label_list.append(synapse_label_dict[synapse])
        label_list_spine.append(synapse_label_dict_spine[synapse])
        joint_list.append(synapse_label_dict[synapse]*100 + synapse_label_dict_spine[synapse])
    label_counter = Counter(label_list)
   
    label_counter_spine = Counter(label_list_spine)
    label_counter_joint = Counter(joint_list)
    #print("label_counter_spine[13]=" + str(label_counter_spine[13]))
    #print(label_counter_spine)
    #print(label_counter)
    
    #the indexes here correspond to the labels b/c it is a dictionary
    apicals   = label_counter[2]
    basals    = label_counter[3]
    obliques  = label_counter[4]
    somas     = label_counter[5]
    axons     = label_counter[6] + label_counter[7]
    
    spines   = label_counter_spine[14]
    spine_heads    = label_counter_spine[13]
    spine_neck  = label_counter_spine[15]
    no_label     = label_counter_spine[0]
    
    apical_joint = (label_counter_joint[214],label_counter_joint[213],label_counter_joint[215],label_counter_joint[200])
    basal_joint = (label_counter_joint[314],label_counter_joint[313],label_counter_joint[315],label_counter_joint[300])
    oblique_joint = (label_counter_joint[414],label_counter_joint[413],label_counter_joint[415],label_counter_joint[400])
    soma_joint = (label_counter_joint[514],label_counter_joint[513],label_counter_joint[515],label_counter_joint[500])
    axon_joint = (label_counter_joint[614] +label_counter_joint[714],
                  label_counter_joint[613] +  label_counter_joint[713],
                  label_counter_joint[615] + label_counter_joint[715],
                  label_counter_joint[600] + label_counter_joint[700])
    
    #print((apicals, basals, obliques, somas, axons))
    #print((apical_joint + basal_joint+ oblique_joint + soma_joint + axon_joint))
    
    
    
    
    #print(np.array((apicals, basals, obliques, somas, axons)))
    synapse_total = sum((apicals, basals, obliques, somas, axons))
    synapse_total_spine = sum((spines, spine_heads, spine_neck, no_label))
    
    if synapse_total > 0:
        axon_to_vector[presyn] = np.array((apicals, basals, obliques, somas, axons))
        axon_to_vector_spine[presyn] = np.array((spines, spine_heads, spine_neck, no_label))
        axon_to_vector_spine_joint[presyn] = np.array(apical_joint + basal_joint
                                                    + oblique_joint + soma_joint + axon_joint)
    
        if synapse_total > 3:
            threshold_axon_to_vector[presyn] = np.array((apicals, basals, obliques, somas, axons))
            threshold_axon_to_vector_spine[presyn] = np.array((spines, spine_heads, spine_neck, no_label))
            threshold_axon_to_vector_spine_joint[presyn] = np.array(apical_joint + basal_joint
                                                    + oblique_joint + soma_joint + axon_joint)
    
    #print(axon_to_vector_spine)
    #break

    
axon_compartment_vector = np.array(list(axon_to_vector.values()))
threshold_axon_compartment_vector = np.array(list(threshold_axon_to_vector.values()))

axon_compartment_vector_spine = np.array(list(axon_to_vector_spine.values()))
threshold_axon_compartment_vector_spine = np.array(list(threshold_axon_to_vector_spine.values()))

axon_compartment_vector_spine_joint = np.array(list(axon_to_vector_spine_joint.values()))
threshold_axon_compartment_vector_spine_joint = np.array(list(threshold_axon_to_vector_spine_joint.values()))

#print(axon_compartment_vector_spine)

#creates the probability distribution of connections by just dividing by total
fingerprints_array = np.zeros(axon_compartment_vector.shape)
for i, axon_vector in tqdm(enumerate(axon_compartment_vector)):
    fingerprints_array[i] = axon_vector / axon_vector.sum()

#creates the probability distribution of connections by just dividing by total
threshold_fingerprints_array = np.zeros(threshold_axon_compartment_vector.shape)
for i, axon_vector in tqdm(enumerate(threshold_axon_compartment_vector)):
    threshold_fingerprints_array[i] = axon_vector / axon_vector.sum()


#creates the probability distribution of connections by just dividing by total
fingerprints_array_spine = np.zeros(axon_compartment_vector_spine.shape)
for i, axon_vector in tqdm(enumerate(axon_compartment_vector_spine)):
    fingerprints_array_spine[i] = axon_vector / axon_vector.sum()

#creates the probability distribution of connections by just dividing by total
threshold_fingerprints_array_spine = np.zeros(threshold_axon_compartment_vector_spine.shape)
for i, axon_vector in tqdm(enumerate(threshold_axon_compartment_vector_spine)):
    threshold_fingerprints_array_spine[i] = axon_vector / axon_vector.sum()
    
#creates the probability distribution of connections by just dividing by total
fingerprints_array_spine_joint = np.zeros(axon_compartment_vector_spine_joint.shape)
for i, axon_vector in tqdm(enumerate(axon_compartment_vector_spine_joint)):
    fingerprints_array_spine_joint[i] = axon_vector / axon_vector.sum()

#creates the probability distribution of connections by just dividing by total
threshold_fingerprints_array_spine_joint = np.zeros(threshold_axon_compartment_vector_spine_joint.shape)
for i, axon_vector in tqdm(enumerate(threshold_axon_compartment_vector_spine_joint)):
    threshold_fingerprints_array_spine_joint[i] = axon_vector / axon_vector.sum()


    
threshold_fingerprints_array_spine


100%|██████████| 73822/73822 [00:05<00:00, 12322.30it/s]
72614it [00:01, 47283.07it/s]
10217it [00:00, 33945.09it/s]
72614it [00:01, 47409.41it/s]
10217it [00:00, 39353.79it/s]
72614it [00:01, 45803.21it/s]
10217it [00:00, 33357.29it/s]


array([[0.        , 0.4       , 0.2       , 0.4       ],
       [0.        , 0.25      , 0.25      , 0.5       ],
       [0.        , 0.5       , 0.25      , 0.25      ],
       ...,
       [0.        , 0.83333333, 0.        , 0.16666667],
       [0.        , 0.5       , 0.25      , 0.25      ],
       [0.        , 0.66666667, 0.        , 0.33333333]])

In [24]:
#visualize using pandas
#print(len(threshold_axon_to_vector.keys()))
#print(len(threshold_fingerprints_array_spine))
compartment_table = pd.DataFrame(threshold_fingerprints_array,
                index=threshold_axon_to_vector.keys(), 
                columns = ["apicals", "basals", "obliques", "somas", "axons"])

compartment_table


,apicals,basals,obliques,somas,axons
54101819,0.400000,0.600000,0.000000,0.000000,0.0000
31523985,0.250000,0.750000,0.000000,0.000000,0.0000
66067511,0.000000,1.000000,0.000000,0.000000,0.0000
58597928,0.250000,0.500000,0.083333,0.166667,0.0000
81551741,0.000000,0.600000,0.000000,0.400000,0.0000
66603984,0.000000,1.000000,0.000000,0.000000,0.0000
78569371,0.117647,0.705882,0.176471,0.000000,0.0000
89052482,0.250000,0.750000,0.000000,0.000000,0.0000
114802131,0.000000,1.000000,0.000000,0.000000,0.0000
107609681,0.000000,0.500000,0.250000,0.250000,0.0000


In [25]:
#getting the axon names for the Fabian analysis
apical_frame = compartment_table.loc[compartment_table['apicals'] >= 0.75]
apical_frame
ids_apical_frame = apical_frame.index
print(ids_apical_frame)
apical_frame


Int64Index([130998989,  98068642, 105608993, 112943282,  95551430, 121451412,
             54364005,  78471805,  85543886,  79163419,
            ...
            127423863, 128305307,  73044581,  67784816, 116483679, 121553624,
            118193064, 118912875,  15517761,  22506964],
           dtype='int64', length=106)


,apicals,basals,obliques,somas,axons
130998989,0.750000,0.000000,0.25,0.00,0.0
98068642,1.000000,0.000000,0.00,0.00,0.0
105608993,0.750000,0.000000,0.25,0.00,0.0
112943282,1.000000,0.000000,0.00,0.00,0.0
95551430,0.750000,0.250000,0.00,0.00,0.0
121451412,0.800000,0.000000,0.20,0.00,0.0
54364005,0.750000,0.000000,0.00,0.25,0.0
78471805,0.888889,0.111111,0.00,0.00,0.0
85543886,0.750000,0.250000,0.00,0.00,0.0
79163419,0.750000,0.250000,0.00,0.00,0.0


In [26]:
#visualize using pandas
#print(len(threshold_axon_to_vector.keys()))
#print(len(threshold_fingerprints_array_spine))
spine_table = pd.DataFrame(threshold_fingerprints_array_spine,
                index=threshold_axon_to_vector.keys(), 
                columns = ["spines", "spine_heads", "spine_neck", "no_label"])

spine_table



,spines,spine_heads,spine_neck,no_label
54101819,0.0,0.400000,0.200000,0.400000
31523985,0.0,0.250000,0.250000,0.500000
66067511,0.0,0.500000,0.250000,0.250000
58597928,0.0,0.500000,0.000000,0.500000
81551741,0.0,0.400000,0.000000,0.600000
66603984,0.0,0.500000,0.000000,0.500000
78569371,0.0,0.647059,0.000000,0.352941
89052482,0.0,1.000000,0.000000,0.000000
114802131,0.0,0.800000,0.000000,0.200000
107609681,0.0,0.750000,0.000000,0.250000


In [27]:
#visualize using pandas

compartment_label_list = ["apicals", "basals", "obliques", "somas", "axons"]
spine_label_list = ["spines", "spine_heads", "spine_neck", "no_label"]
joint_label_list = []
for c in compartment_label_list:
    for s in spine_label_list:
        joint_label_list.append(c + ":" + s)

#print(joint_label_list)

spine_joint_table = pd.DataFrame(threshold_fingerprints_array_spine_joint,
                index=threshold_axon_to_vector_spine_joint.keys(), 
                columns = joint_label_list)

spine_joint_table

,apicals:spines,apicals:spine_heads,apicals:spine_neck,apicals:no_label,basals:spines,basals:spine_heads,basals:spine_neck,basals:no_label,obliques:spines,obliques:spine_heads,obliques:spine_neck,obliques:no_label,somas:spines,somas:spine_heads,somas:spine_neck,somas:no_label,axons:spines,axons:spine_heads,axons:spine_neck,axons:no_label
54101819,0.0,0.200000,0.00,0.200000,0.0,0.200000,0.200000,0.200000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
31523985,0.0,0.000000,0.25,0.000000,0.0,0.250000,0.000000,0.500000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
66067511,0.0,0.000000,0.00,0.000000,0.0,0.500000,0.250000,0.250000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
58597928,0.0,0.250000,0.00,0.000000,0.0,0.166667,0.000000,0.333333,0.0,0.000000,0.00,0.083333,0.0,0.083333,0.000000,0.083333,0.0,0.0000,0.00,0.00
81551741,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.000000,0.400000,0.0,0.000000,0.00,0.000000,0.0,0.200000,0.000000,0.200000,0.0,0.0000,0.00,0.00
66603984,0.0,0.000000,0.00,0.000000,0.0,0.500000,0.000000,0.500000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
78569371,0.0,0.000000,0.00,0.117647,0.0,0.470588,0.000000,0.235294,0.0,0.176471,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
89052482,0.0,0.250000,0.00,0.000000,0.0,0.750000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
114802131,0.0,0.000000,0.00,0.000000,0.0,0.800000,0.000000,0.200000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000,0.00,0.00
107609681,0.0,0.000000,0.00,0.000000,0.0,0.250000,0.000000,0.250000,0.0,0.250000,0.00,0.000000,0.0,0.250000,0.000000,0.000000,0.0,0.0000,0.00,0.00


In [28]:
###############  THE CONDENSED JOINT GROUPINGS ####################

#combine columns: so only heads and non-heads
#create mapping
compartment_label_list = ["apicals", "basals", "obliques", "somas", "axons"]
spine_label_list = ["spine_heads","spine_neck", "no_label"]
drop_spine_list = ["spines"]


mapping_merge = dict()
for i in compartment_label_list:
    label = i + ":non_head"
    for j in spine_label_list:
        if j == "spine_heads":
            mapping_merge[i + ":" + j] = i + ":" + j
        else:
            mapping_merge[i + ":" + j] = label
        
#print(mapping_merge)
        

spine_head_no_head_table = spine_joint_table.groupby(mapping_merge,axis=1).sum()


spine_head_no_head_table

,apicals:non_head,apicals:spine_heads,axons:non_head,axons:spine_heads,basals:non_head,basals:spine_heads,obliques:non_head,obliques:spine_heads,somas:non_head,somas:spine_heads
54101819,0.200000,0.200000,0.00,0.0000,0.400000,0.200000,0.000000,0.000000,0.000000,0.000000
31523985,0.250000,0.000000,0.00,0.0000,0.500000,0.250000,0.000000,0.000000,0.000000,0.000000
66067511,0.000000,0.000000,0.00,0.0000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000
58597928,0.000000,0.250000,0.00,0.0000,0.333333,0.166667,0.083333,0.000000,0.083333,0.083333
81551741,0.000000,0.000000,0.00,0.0000,0.400000,0.200000,0.000000,0.000000,0.200000,0.200000
66603984,0.000000,0.000000,0.00,0.0000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000
78569371,0.117647,0.000000,0.00,0.0000,0.235294,0.470588,0.000000,0.176471,0.000000,0.000000
89052482,0.000000,0.250000,0.00,0.0000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000
114802131,0.000000,0.000000,0.00,0.0000,0.200000,0.800000,0.000000,0.000000,0.000000,0.000000
107609681,0.000000,0.000000,0.00,0.0000,0.250000,0.250000,0.000000,0.250000,0.000000,0.250000


In [29]:
compartment_table

,apicals,basals,obliques,somas,axons
54101819,0.400000,0.600000,0.000000,0.000000,0.0000
31523985,0.250000,0.750000,0.000000,0.000000,0.0000
66067511,0.000000,1.000000,0.000000,0.000000,0.0000
58597928,0.250000,0.500000,0.083333,0.166667,0.0000
81551741,0.000000,0.600000,0.000000,0.400000,0.0000
66603984,0.000000,1.000000,0.000000,0.000000,0.0000
78569371,0.117647,0.705882,0.176471,0.000000,0.0000
89052482,0.250000,0.750000,0.000000,0.000000,0.0000
114802131,0.000000,1.000000,0.000000,0.000000,0.0000
107609681,0.000000,0.500000,0.250000,0.250000,0.0000


In [30]:
#Do thresholding example for high percentage in each category

apical_axons = list(compartment_table.loc[compartment_table['apicals'] >= 0.75].index)
basal_axons = list(compartment_table.loc[compartment_table['basals'] >= 0.75].index)
oblique_axons = list(compartment_table.loc[compartment_table['obliques'] >= 0.75].index)
soma_axons = list(compartment_table.loc[compartment_table['somas'] >= 0.75].index)
axon_axons = list(compartment_table.loc[compartment_table['axons'] >= 0.75].index)

print(type(list(axon_axons)))


other_axons = list(set(list(compartment_table.index)).difference(set(apical_axons + basal_axons + oblique_axons + soma_axons + axon_axons)))
print(len(compartment_table.index))
total_axons = compartment_table.index

total_length = len(apical_axons) + len(basal_axons) + len(oblique_axons) + len(soma_axons) + len(axon_axons) + len(other_axons)
print(total_length)

print(len(other_axons))

np.savez("compartment_axon_groups.npz",
         apical_axons=apical_axons,
        basal_axons=basal_axons,
        oblique_axons=oblique_axons,
        soma_axons=soma_axons,
        axon_axons=axon_axons,
        other_axons=other_axons,
        total_axons=total_axons)





<class 'list'>
10217
10217
4605


In [31]:
Label_Table

,synapse_id,presyn,postsyn,postsynaptic_label,spine_numeric,description,spine_description
0,21460,54101819,36456160,3,0,Basal,not_labeled
1,21485,50165577,36456160,3,14,Basal,Spine
2,23369,58116980,36456160,3,15,Basal,Spine Neck
3,23579,54037914,36456160,3,15,Basal,Spine Neck
4,23689,54101819,36456160,3,0,Basal,not_labeled
5,23728,54034799,36456160,3,13,Basal,Spine Head
6,23805,58036847,36456160,3,13,Basal,Spine Head
7,23870,50165937,36456160,3,0,Basal,not_labeled
8,25181,58037816,36456160,3,14,Basal,Spine
9,25797,31523985,36456160,3,14,Basal,Spine


In [32]:
#compile the list
axon_groups = [apical_axons,basal_axons,oblique_axons,soma_axons,axon_axons,other_axons,total_axons]
axon_names = ["apical (>0.75)","basal (>0.75)","oblique (>0.75)","soma (>0.75)","axon (>0.75)","other","total"]

#pull down all of the 
postsyn_list = Label_Table.postsyn.unique()
#print(postsyn_list)
ax_group_dicts = {}

final_table = [None] * (len(postsyn_list)*len(postsyn_list) - len(postsyn_list))*len(axon_names)
counter = 0
for ax_group,ax_name in tqdm(zip(axon_groups,axon_names)):
    postsyn_axon_dict = {k:set() for k in postsyn_list}
    
    #print(postsyn_axon_dict)
    for ax in ax_group:
        #get the postsynaptic targets
        #print(ax)
        #print(Label_Table["presyn"])
        mini_table = Label_Table.loc[Label_Table["presyn"] == ax]
        if len(mini_table) > 0:
            #print(mini_table)
            unique_postsyns = mini_table.postsyn.unique()
        else:
            continue
            
        for up in unique_postsyns:
            postsyn_axon_dict[up].add(ax)
    
    ax_group_dicts[ax_name] = postsyn_axon_dict
     
#ax_group_dicts
counter = 0
#generate the rows for the table
for ax_group_name, ax_group_list in tqdm(ax_group_dicts.items()):
    #print("ax_group_name = " + str(ax_group_name))
    #print("ax_group_list = " + str(ax_group_list))
    for post,post_list in ax_group_list.items():
        for pos_post in postsyn_list:
            if pos_post == post:
                continue
            n_seg = len(post_list.intersection(ax_group_dicts[ax_group_name][pos_post]))
            #print(n_seg)
            final_table[counter] = (post,ax_group_name,pos_post,n_seg)
            #print(final_table)
            counter += 1
            
#print(final_table[(len(postsyn_list)*len(postsyn_list) - len(postsyn_list))-1])
table_without_functional = pd.DataFrame.from_records(final_table,columns= ["segment_a","connection","segment_b","n_seg_shared"])
    


            
    
    
    


7it [00:26,  5.13s/it]
100%|██████████| 7/7 [00:00<00:00,  5.38it/s]


In [33]:
table_without_functional.loc[table_without_functional["connection"]=="other"]
#print(final_table[(len(postsyn_list)*len(postsyn_list) - len(postsyn_list))*5-1])
#table_without_functional

,segment_a,connection,segment_b,n_seg_shared
138610,36456160,other,76859794,5
138611,36456160,other,33376321,2
138612,36456160,other,83286327,9
138613,36456160,other,36706215,1
138614,36456160,other,24927722,2
138615,36456160,other,31978704,3
138616,36456160,other,45965481,1
138617,36456160,other,34028137,0
138618,36456160,other,46354569,3
138619,36456160,other,101666914,2


In [34]:
#sets up the graphing variables for below
is_tuned = 'von_r2>0.05 and von_pvalue<0.05'
dori_resolution = np.pi / 2 / 4
rad2deg = 180/np.pi
bin_edges = np.linspace(0, np.pi,9)
be = list(['{:.0f}'.format(ee) for ee in [np.round(e * rad2deg) for e in bin_edges]])
bin_labels = list(zip(be[:-1], be[1:]))
bin_centers = np.round((bin_edges[1:] + bin_edges[:-1])/2 * rad2deg, decimals=2) 

In [35]:
tuned_rel = nda.VonMises() & is_tuned
best_r2 = dj.U('segment_id').aggr(tuned_rel, best_r2='max(von_r2)')
pref_rel = (tuned_rel * best_r2) & 'von_r2 >= best_r2'
pref_rel


scan_idx scan ID,segmentation segmentation id,segment_id segment id unique within each Segmentation,von_r2 r-squared explaned by vonMises fit,von_pref preferred directions,von_base von mises base value,von_amp1 amplitude of first peak,von_amp2 amplitude of second peak,von_sharp sharpnesses,von_pvalue p-value by shuffling (nShuffles = 1e4),best_r2 calculated attribute
2,1,2589588,0.053372,3.02107,0.00165716,0.0787312,0.0374586,26.5923,0.006497,0.053372
2,1,3813916,0.0578909,2.90859,0.017539,0.139912,0.0,16.2428,0.0014995,0.0578909
2,1,10914339,0.0799213,3.13328,0.0113467,0.297867,0.169138,11.2455,0.002499,0.0799213
2,1,16809519,0.236849,4.00482,0.00729216,0.537001,0.32923,19.5817,0.0005,0.236849
2,1,23410728,0.118081,2.10778,0.0550229,1.0047,0.292185,5.85615,0.0005,0.118081
2,1,27229091,0.161084,5.57486,0.00616441,0.450609,0.0902593,14.4957,0.0005,0.161084
2,1,38469670,0.198155,5.51238,0.00350638,0.232938,0.0834227,37.8783,0.0005,0.198155


In [36]:
#add on the functional group preferences
#restricts and appends the orientation

#query that will get the functional data
rel2 = pref_rel.proj('scan_idx',ori_1='von_pref', segment_a='segment_id') 
functional_id,ori,scan_idx = rel2.fetch("segment_a","ori_1","scan_idx") 

#create functional lookup table
func_lookup = dict()
for i,id in enumerate(functional_id):
    func_lookup[id] = [ori[i],scan_idx[i]]

#iterate through the list and only keep those whose segments are in functional list and then append the data
final_list_with_functional = []#["None"] * len(final_table)
counter = 0

dummy=2
segmentation=1


for ll in final_table: 
    if ll[0] in functional_id and ll[2] in functional_id:
        final_list_with_functional.append((ll[0],ll[1],ll[2],ll[3],
                                               int(dummy),int(segmentation),int(func_lookup[ll[0]][1]),func_lookup[ll[0]][0],
                                              func_lookup[ll[2]][0]))
        counter += 1

print(len(final_list_with_functional))

4550


In [37]:
#print(final_table[(len(postsyn_list)*len(postsyn_list) - len(postsyn_list))-1])
table_with_functional = pd.DataFrame.from_records(final_list_with_functional,columns= 
                ["segment_a","connection","segment_b","n_seg_shared","dummy","segmentation","scan_idx",
                        "ori_1","ori_2"])

#table_with_functional.loc[table_with_functional["n_seg_shared"]>0]
###########--------------------TABLE TO USE FOR THE FABIAN ANALYSIS------------------##########
table_with_functional

,segment_a,connection,segment_b,n_seg_shared,dummy,segmentation,scan_idx,ori_1,ori_2
0,83286327,apical (>0.75),46354569,0,2,1,2,5.463940,3.851180
1,83286327,apical (>0.75),10356289,0,2,1,2,5.463940,3.174130
2,83286327,apical (>0.75),39500491,0,2,1,2,5.463940,3.106100
3,83286327,apical (>0.75),942272,0,2,1,2,5.463940,5.087230
4,83286327,apical (>0.75),23683777,0,2,1,2,5.463940,2.921270
5,83286327,apical (>0.75),78665518,0,2,1,2,5.463940,2.973850
6,83286327,apical (>0.75),94050687,0,2,1,2,5.463940,2.239210
7,83286327,apical (>0.75),107731759,0,2,1,2,5.463940,4.128270
8,83286327,apical (>0.75),67981253,0,2,1,2,5.463940,4.891510
9,83286327,apical (>0.75),107860456,0,2,1,2,5.463940,3.164700


In [38]:
table_with_functional.to_pickle("shared_connections.pkl")

In [39]:
'''# import tsne
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import gaussian_kde
from numpy import linalg
import seaborn as sns
import random

#Try an example clustering
input_axons = threshold_fingerprints_array_spine_joint 
clusterer_input = input_axons
clusterer = KMeans(n_clusters=8, n_init=10, random_state=0)
clusterer.fit(clusterer_input)
clusterer.labels_

scores_per_cluster = np.zeros((input_axons.shape[0], clusterer.n_clusters))
for i, feature_vector in enumerate(input_axons):
    for j, center in enumerate(clusterer.cluster_centers_):
        scores_per_cluster[i, j] = np.linalg.norm(center-feature_vector)
scores_per_cluster = np.exp(-scores_per_cluster)
# scores_per_cluster = scores_per_cluster
groupings = clusterer.labels_#np.argmin(scores_per_cluster, axis=1)
groupings_dict = dict()
for i in np.unique(groupings):
    groupings_dict[i] = list()
for i, cluster in enumerate(groupings):
    groupings_dict[cluster].append(i)
mean_scores_per_group = np.zeros((clusterer.n_clusters, clusterer.n_clusters))
for i, axons in groupings_dict.items():
    group_scores = scores_per_cluster[np.array(axons)]
    for j, row in enumerate(group_scores):
        group_scores[j] /= row.sum()
    for j, column in enumerate(group_scores.T):
        mean_scores_per_group[i, j] = column.mean()
for i, row in enumerate(mean_scores_per_group):
    print("Expected Cluster:", i, "-> index of max:", np.argmax(row))
sns.heatmap(mean_scores_per_group)

'''

'# import tsne\nfrom sklearn.decomposition import PCA\nfrom sklearn.cluster import KMeans\nfrom scipy.stats import gaussian_kde\nfrom numpy import linalg\nimport seaborn as sns\nimport random\n\n#Try an example clustering\ninput_axons = threshold_fingerprints_array_spine_joint \nclusterer_input = input_axons\nclusterer = KMeans(n_clusters=8, n_init=10, random_state=0)\nclusterer.fit(clusterer_input)\nclusterer.labels_\n\nscores_per_cluster = np.zeros((input_axons.shape[0], clusterer.n_clusters))\nfor i, feature_vector in enumerate(input_axons):\n    for j, center in enumerate(clusterer.cluster_centers_):\n        scores_per_cluster[i, j] = np.linalg.norm(center-feature_vector)\nscores_per_cluster = np.exp(-scores_per_cluster)\n# scores_per_cluster = scores_per_cluster\ngroupings = clusterer.labels_#np.argmin(scores_per_cluster, axis=1)\ngroupings_dict = dict()\nfor i in np.unique(groupings):\n    groupings_dict[i] = list()\nfor i, cluster in enumerate(groupings):\n    groupings_dict[cl

In [40]:

#print the average compartment level distribution of each grouping
import numpy as np
cluster_output = clusterer.labels_

group_distributions = dict()

for i,ax in enumerate(input_axons):
    #get grouping
    group = cluster_output[i]
    if group not in group_distributions:
        group_distributions[group] = list()
        
    group_distributions[group].append(ax)

print()
#find the average distribution for each group
average_dist = [np.matrix(k).mean(axis=0) for i,k in group_distributions.items()]
variance_dist = [np.matrix(k).var(axis=0) for i,k in group_distributions.items()]
group_population = [len(k) for i,k in group_distributions.items()]

print(group_population)
print("average_dist = ")
print("apicals,     basals,   obliques,    somas,  axons")
for i in average_dist:
    print(i)
    

print("variance_dist = ")
print("apicals,     basals,   obliques,    somas,  axons")
for i in variance_dist:
    print(i)


NameError: name 'clusterer' is not defined

# Fabian Analysis Code

In [ ]:
import funconnect